###Importing libraries

In [1]:
import numpy as np
import pandas as pd

###Importing dataset (from google drive)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd


In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/2 Sentimental analysis
!ls

/content/drive/MyDrive/Colab Notebooks/2 Sentimental analysis
 c1_BoW_Sentiment_Model.pkl			 'Copy of a2_RestaurantReviews_FreshDump.tsv'
 c2_Classifier_Sentiment_Model			  NoQsgooglereviews.tsv
 c3_Predicted_Sentiments_Fresh_Dump.tsv		  Sentiment_Analysis1.ipynb
 classifier.pkl					  Sentiment_Predictor.ipynb
'Copy of a1_RestaurantReviews_HistoricDump.tsv'   vectorizer.pkl
'Copy of a2_RestaurantReviews_FreshDump.gsheet'


In [5]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/2 Sentimental analysis/Copy of a2_RestaurantReviews_FreshDump.tsv', delimiter = '\t', quoting = 3)
dataset.head()

,Review
0,Spend your money elsewhere.
1,Their regular toasted bread was equally satisf...
2,The Buffet at Bellagio was far from what I ant...
3,"And the drinks are WEAK, people!"
4,-My order was not correct.


###Data cleaning

In [6]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Initialize objects
ps = PorterStemmer()
all_stopwords = set(stopwords.words('english'))

# Ensure corpus is empty
corpus = []

# Loop through dataset
for i in range(min(100, len(dataset))):  # Avoid index out of range error
    review = str(dataset['Review'][i])  # Convert to string
    if review == "nan":  # Handle NaN values
        review = ""

    review = re.sub('[^a-zA-Z]', ' ', review)  # Remove non-alphabet characters
    review = review.lower().split()
    review = [ps.stem(word) for word in review if word not in all_stopwords]
    review = ' '.join(review)

    corpus.append(review)  # Append to corpus


###Data transformation

In [8]:
# Loading BoW dictionary
from sklearn.feature_extraction.text import CountVectorizer
import pickle
cvFile='/content/drive/MyDrive/Colab Notebooks/2 Sentimental analysis/c1_BoW_Sentiment_Model.pkl'
# cv = CountVectorizer(decode_error="replace", vocabulary=pickle.load(open('./drive/MyDrive/Colab Notebooks/2 Sentiment Analysis (Basic)/3.1 BoW_Sentiment Model.pkl', "rb")))
cv = pickle.load(open(cvFile, "rb"))


In [9]:
import pickle

# Load the pickle file properly
with open("/content/drive/MyDrive/Colab Notebooks/2 Sentimental analysis/c1_BoW_Sentiment_Model.pkl", "rb") as file:
    dataset = pickle.load(file)

print(type(dataset))  # Check what type of object it is



<class 'sklearn.feature_extraction.text.CountVectorizer'>


###Predictions (via sentiment classifier)

In [10]:
import joblib
classifier = joblib.load('c2_Classifier_Sentiment_Model')

In [11]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer

import pickle

# Load the pickle file properly
with open("/content/drive/MyDrive/Colab Notebooks/2 Sentimental analysis/c1_BoW_Sentiment_Model.pkl", "rb") as file:
    model = pickle.load(file)

print(type(model))  # Check the type of the loaded object



<class 'sklearn.feature_extraction.text.CountVectorizer'>


In [12]:
import pickle

# Load the pickle file correctly
with open("/content/drive/MyDrive/Colab Notebooks/2 Sentimental analysis/c1_BoW_Sentiment_Model.pkl", "rb") as file:
    model = pickle.load(file)  # This can be a dictionary, classifier, or vectorizer

print(type(model))  # Check what type of object is inside



<class 'sklearn.feature_extraction.text.CountVectorizer'>


In [13]:
import pickle
import pandas as pd

# Load the trained classifier
with open("/content/drive/MyDrive/Colab Notebooks/2 Sentimental analysis/classifier.pkl", "rb") as file:
    classifier = pickle.load(file)

# Load the pre-trained CountVectorizer
with open("/content/drive/MyDrive/Colab Notebooks/2 Sentimental analysis/vectorizer.pkl", "rb") as file:
    cv = pickle.load(file)

# Load the dataset
dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2 Sentimental analysis/Copy of a1_RestaurantReviews_HistoricDump.tsv",
                       delimiter="\t", quoting=3)

# Convert text to list and ensure it's properly formatted
corpus = dataset.iloc[:, 0].astype(str).tolist()

# **Important**: Transform text using the SAME vectorizer that was trained
X_fresh = cv.transform(corpus)  # Don't fit again, just transform!

# Ensure feature consistency
print("Expected features:", classifier.n_features_in_)
print("New data features:", X_fresh.shape[1])

if X_fresh.shape[1] != classifier.n_features_in_:
    print("Feature mismatch! Ensure you are using the correct vectorizer.")
else:
    # Make predictions
    y_pred = classifier.predict(X_fresh.toarray())

    # Add predictions to dataset
    dataset["predicted_label"] = y_pred.tolist()

    # Save output file
    dataset.to_csv("/content/drive/MyDrive/Colab Notebooks/2 Sentimental analysis/c3_Predicted_Sentiments_Fresh_Dump.tsv",
                   sep="\t", encoding="utf-8", index=False)

    print("Prediction file saved successfully!")



Expected features: 1420
New data features: 171
Feature mismatch! Ensure you are using the correct vectorizer.
